In [ ]:
# Standard Python Modules:
import os
from datetime import datetime


# Data Wrangling:
import numpy as np
import pandas as pd


# Machine Learning:

# Feature/Training Enhancements:
from sklearn.impute import KNNImputer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split


# Models:
from sklearn.ensemble import RandomForestRegressor


# Model Performance Metrics:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import explained_variance_score


# Deep Learning:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
train = pd.read_csv("train.csv")
train.columns = [feature_name.upper() for feature_name in train.columns]
train.FIRST_DAY_OF_MONTH = pd.to_datetime(train.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
train["YEAR"] = train.FIRST_DAY_OF_MONTH.dt.year
train["MONTH"] = train.FIRST_DAY_OF_MONTH.dt.month
train["STATE_FIPS_CODE"] = train.CFIPS.astype(str).apply(lambda x: x[:2])
train["COUNTY_FIPS_CODE"] = train.CFIPS.astype(str).apply(lambda x: x[1:])
del (
    train["COUNTY"],
    train["STATE"],
    train["ROW_ID"],
    train["ACTIVE"],
    train["FIRST_DAY_OF_MONTH"],
)
print(train.shape, "(ROWS, COLUMNS)")
train.head(3)

In [ ]:
test = pd.read_csv("test.csv")
test.columns = [feature_name.upper() for feature_name in test.columns]
test.FIRST_DAY_OF_MONTH = pd.to_datetime(test.FIRST_DAY_OF_MONTH, format="%Y-%m-%d")
test["YEAR"] = test.FIRST_DAY_OF_MONTH.dt.year
test["MONTH"] = test.FIRST_DAY_OF_MONTH.dt.month
test["STATE_FIPS_CODE"] = test.CFIPS.astype(str).apply(lambda x: x[:2])
test["COUNTY_FIPS_CODE"] = test.CFIPS.astype(str).apply(lambda x: x[1:])
del test["ROW_ID"], test["FIRST_DAY_OF_MONTH"]
print(test.shape, "(ROWS, COLUMNS)")
test.head(3)

In [ ]:
census_starter = pd.read_csv("census_starter.csv")
census_starter.columns = [
    feature_name.upper() for feature_name in census_starter.columns
]
census_starter.head(3)

In [ ]:
imputed_census_starter = pd.DataFrame(
    KNNImputer(n_neighbors=5).fit_transform(census_starter),
    columns=list(census_starter.columns),
)
imputed_census_starter.head(3)

In [ ]:
pd.concat(
    [
        census_starter.mean().rename("ORIGINAL_MEAN"),
        imputed_census_starter.mean().rename("IMPUTED_MEAN"),
        census_starter.std().rename("ORIGINAL_SD"),
        imputed_census_starter.std().rename("IMPUTED_SD"),
    ],
    axis=1,
)

In [ ]:
del census_starter

In [ ]:
print(train.shape)
train = pd.merge(left=train, right=imputed_census_starter, on=["CFIPS"], how="inner")
print(train.shape)
train.head(3)

In [ ]:
print(test.shape)
test = pd.merge(left=test, right=imputed_census_starter, on=["CFIPS"], how="inner")
print(test.shape)
test.head(3)

In [ ]:
train.STATE_FIPS_CODE.nunique(), test.STATE_FIPS_CODE.nunique()

In [ ]:
print(train.shape)
train = pd.concat(
    [
        train,
        pd.get_dummies(
            train["STATE_FIPS_CODE"], prefix="STATE_FIPS_CODE_", drop_first=True
        ),
    ],
    axis=1,
)
del train["STATE_FIPS_CODE"]
print(train.shape)

In [ ]:
print(test.shape)
test = pd.concat(
    [
        test,
        pd.get_dummies(
            test["STATE_FIPS_CODE"], prefix="STATE_FIPS_CODE_", drop_first=True
        ),
    ],
    axis=1,
)
del test["STATE_FIPS_CODE"]
print(test.shape)

In [ ]:
feature_remove_list = ["CFIPS", "MICROBUSINESS_DENSITY"]

X = train[[f for f in train.columns if f not in feature_remove_list]]
y = train["MICROBUSINESS_DENSITY"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42
)

In [ ]:
random_forest_estimator = RandomForestRegressor(
    n_estimators=2,
    criterion="squared_error",
    max_depth=4,
    min_samples_split=0.01,
    min_samples_leaf=0.09,
    min_weight_fraction_leaf=0.0,
    max_features=1.0,
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    bootstrap=True,
    oob_score=False,
    n_jobs=-1,
    random_state=1024,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
).fit(X_train, y_train)

In [ ]:
training_metrics = pd.DataFrame({
    'ACTUAL': y_train,
    'PREDICTED': random_forest_estimator.predict(X_train)
})

test_metrics = pd.DataFrame({
    'ACTUAL': y_test,
    'PREDICTED': random_forest_estimator.predict(X_test)
})

In [ ]:
print(
    'EXPLAINED_VARIANCE_TRAIN:', 
    explained_variance_score(training_metrics['ACTUAL'], training_metrics['PREDICTED']),
    '\nEXPLAINED_VARIANCE_TEST:', 
    explained_variance_score(test_metrics['ACTUAL'], test_metrics['PREDICTED'])
)

In [ ]:
print(
    'MSE:', 
    mean_squared_error(training_metrics['ACTUAL'], training_metrics['PREDICTED']),
    '\nMSE:', 
    mean_squared_error(test_metrics['ACTUAL'], test_metrics['PREDICTED'])
)

In [24]:
submission = pd.read_csv('test.csv')
submission['microbusiness_density'] = random_forest_estimator.predict(test[[f for f in test.columns if f!='CFIPS']])
del submission['cfips'], submission['first_day_of_month']
submission.to_csv('rf_submission_02.csv', index=False)
submission.head(3)

row_id  microbusiness_density
0  1001_2022-11-01               4.654223
1  1003_2022-11-01               4.654223
2  1005_2022-11-01               4.654223

In [ ]:
model = Sequential()
model.add(Dense(100, X_train.shape[0], activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(1, activation='linear'))
model.summary()

In [ ]:
model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [ ]:
history = model.fit(X_train, y_train, epochs=150, batch_size=50,  verbose=1, validation_split=0.3)

In [ ]:
print(history.history.keys())
# "Loss"
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

A KAP (Knowledge, Attitude, Practice) survey is a type of survey that is used to assess people's knowledge, attitudes, and practices related to a specific topic or issue. Here is a general overview of the steps involved in analyzing a KAP survey:

    Check the response rate: Look at the response rate to see how many people completed the survey out of the total number of people who were invited to participate. A high response rate indicates that the sample is likely to be representative of the population.

    Review the survey questions: Review the survey questions to make sure that they are clear and relevant to the topic of the survey.

    Prepare the data: Prepare the data by cleaning and coding it, so that it is ready for analysis.

    Summarize the data: Summarize the data by calculating descriptive statistics such as means, frequencies, and percentages. This will give you an overall picture of the data.

    Analyze the data: Analyze the data using appropriate statistical tests such as chi-square or t-tests to identify any significant differences between subgroups or to test hypotheses.

    Interpret the results: Interpret the results by comparing them to the objectives of the survey and the relevant literature. Also, consider the limitations of the survey and the potential impact of any biases or confounding variables.

    Communicate the findings: Communicate the findings by creating tables, figures, and reports that present the results in a clear and concise way.